# Lab-10-6-1 Advanced CNN(RESNET)1

In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [2]:
def conv3x3(in_planes, out_planes, stride=1):
  '''3x3 convolution with padding'''
  return nn.Conv2d(in_planes, out_planes, kernel_size = 3, stride = stride,
                   padding = 1, bias = True)
  
def conv1x1(in_planes, out_planes, stride = 1):
  ''' 1x1 convolution '''
  return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride= stride, bias = False)

In [3]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, inplanes, planes, stride=1, downsample = None):
    super(BasicBlock, self).__init__()
    self.conv1 = conv3x3(inplanes, planes, stride)
    self.bn1 = nn.BatchNorm2d(planes)
    self.relu = nn.ReLU(inplace = True)
    self.conv2 = conv3x3(planes, planes)
    self.bn2 = nn.BatchNorm2d(planes)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):

    ''' stride 2라고 가정'''
    # x.shape = 3 x 64 x 64
    identity = x
    # identity = 3 x 64 x 64
    out = self.conv1(x) # 3 x 3 stride = stride
    # out.shape = 3 x 32 x 32
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out) # 3 x 3 stride = 1
    out = self.bn2(out)

    # out.shape = 3 x 32 x 32
    # identity = 3 x 64 x 64
    # 덧셈 불가 
    # => downsample 사용
    if self.downsample is not None:
      identity = self.downsample(x)   # stride 가 2인 경우에 문제가 생김
      # stride가 2인 경우는 주석으로 설명 처리 하겠음.

    out += identity
    out = self.relu(out)


    return out

In [4]:
class Bottleneck(nn.Module):
  expansion = 4

  def __init__(self, inplanes, planes, stride = 1, downsample = None):
    super(Bottleneck, self).__init__()
    self.conv1 = conv1x1(inplanes, planes) # conv1x1(64, 64)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = conv3x3(planes, planes, stride) # conv3x3(64, 64)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = conv1x1(planes, planes * self.expansion) # conv1x1(64, 256)

    self.bn3 = nn.BatchNorm2d(planes * self.expansion)
    self.relu = nn.ReLU(inplace = True)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):
    identity = x
  
    out = self.conv1(x) # 1x1 stride = 1
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)  # 3x3 stride = stride
    out = self.bn2(out)
    out = self.relu(out)

    out = self.conv3(out)   # 1x1 planes, planes * self.expansion
    out = self.bn3(out)

    if self.downsample is not None:
      identity = self.downsample(x)

    out += identity
    out = self.relu(out)


    return out

In [5]:
class ResNet(nn.Module):
  # model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) # resnet 50

  def __init__(self, block, layers, num_classes = 1000, zero_init_residual=False):
    super(ResNet, self).__init__()

    self.inplanes = 64
    # inputs = 3x224x224
    
    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride =2, padding = 3, bias = False)
    # outputs = self.conv1(inputs)
    # outputs.shape = [64x112x112]
    
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace= True)

    # inputs = [64x112x112]
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride =2 ,padding = 1)
    # outputs = [64, 56, 56]

    self.layer1 = self._make_layer(block, 64, layers[0]) # 3
    self.layer2 = self._make_layer(block, 128, layers[1], stride = 2) # 4
    self.layer3 = self._make_layer(block, 256, layers[2], stride = 2) # 6
    self.layer4 = self._make_layer(block, 512, layers[3], stride = 2) # 3

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512 * block.expansion, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')

      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


      # Zero_initialze the last BN in each residual branch,
      # so that the residual branch starts with zeros, and each residual
      # block behaves like an identity
      # This improves the model by 0.2~0.3% according th https://arxiv.org/abs/1706.02677

      if zero_init_residual:
        for m in self.modules():
          if isinstance(m, Bottleneck):
            nn.init.constant_(m.bn3.weight, 0)
          elif isinstance(m, BasicBlock):
            nn.init.constant_(m.bn2.weight, 0)

  # self.inplanes = 64          
  # self.layer1 = self._make_layer(block, 64, layers[0]) # 3
  def _make_layer(self, block, planes, blocks, stride = 1):
    downsample = None

    if stride != 1 or self.inplanes != planes * block.expansion:

      downsample = nn.Sequential(
          conv1x1(self.inplanes, planes * block.expansion, stride), # conv1x1(256, 512, 2)
          nn.BatchNorm2d(planes * block.expansion), # batchnorm2d(512)
      )

    layers = []
    layers.append(block(self.inplanes, planes, stride, downsample))
    # layers.append(BottleNeck(64, 64, 1, downsample))

    self.inplanes = planes * block.expansion # self.inplanes = 256

    for _ in range(1, blocks):
      layers.append(block(self.inplanes, planes)) # * 2

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)

    return x

In [6]:
def resnet18(pretrained= False, **kwargs):
  model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs) # => 2*(2+2+2+2) +1(conv1) +1(fc) = 16 + 2 = resnet 18
  return model

In [7]:
def resnet50(pretrained=False, **kwargs):
  model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) # => 3*(3+4+6+3) +1(conv1) +1(fc) = 48 + 2 = 50
  return model

In [8]:
def resnet152(pretrained=False, **kwargs):
  model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs) # 3*(3+8+36+3) + 2 = 150 + 2 = 152
  return model

In [9]:
res = resnet50()

In [10]:
res

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fal

# Lab-10-6-2 Advanced CNN(RESNET)2

In [11]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

cuda


In [13]:
transform = transforms.Compose([
                                 transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train = True, download = True, transform = transform)

print(trainset.data.shape)

train_data_mean = trainset.data.mean(axis=(0, 1, 2))
train_data_std = trainset.data.std(axis=(0,1,2))

print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 25

print(train_data_mean)
print(train_data_std)



Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[2.51972877 2.48354831 2.66819599]


In [14]:
transform_train = transforms.Compose([
                                      transforms.RandomCrop(32, padding = 4), 
                                      # padding을 4로 둘러주고 거기서 32만큼 랜덤하게 뜯어 오겠다.
                                      # dataset을 여러가지로 (특이한 형태로) 가지고 올 수 있게함.
                                      transforms.ToTensor(),
                                      transforms.Normalize(train_data_mean, train_data_std)

])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(train_data_mean, train_data_std)
])

trainset = torchvision.datasets.CIFAR10(root = './cifar10', train = True,
                                        download = True, transform = transform_train)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 256, 
                                          shuffle = True, num_workers = 0)

testset = torchvision.datasets.CIFAR10(root = './cifar10', train = False,
                                        download = True, transform = transform_test)

testloader = torch.utils.data.DataLoader(trainset, batch_size = 256, 
                                          shuffle = False, num_workers = 0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [15]:
class ResNet(nn.Module):
  # model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) # resnet 50

  def __init__(self, block, layers, num_classes = 1000, zero_init_residual=False):
    super(ResNet, self).__init__()

    self.inplanes = 16
    # inputs = 3x224x224
    
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride =1, padding = 1, bias = False)
    # outputs = self.conv1(inputs)
    # outputs.shape = [64x112x112]
    
    self.bn1 = nn.BatchNorm2d(16)
    self.relu = nn.ReLU(inplace= True)

    # inputs = [64x112x112]
    # self.maxpool = nn.MaxPool2d(kernel_size = 3, stride =2 ,padding = 1)
    # outputs = [64, 56, 56]

    self.layer1 = self._make_layer(block, 16, layers[0], stride = 1) # 3
    self.layer2 = self._make_layer(block, 32, layers[1], stride = 1) # 4
    self.layer3 = self._make_layer(block, 64, layers[2], stride = 2) # 6
    self.layer4 = self._make_layer(block, 128, layers[3], stride = 2) # 3

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(128 * block.expansion, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')

      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


      # Zero_initialze the last BN in each residual branch,
      # so that the residual branch starts with zeros, and each residual
      # block behaves like an identity
      # This improves the model by 0.2~0.3% according th https://arxiv.org/abs/1706.02677

      if zero_init_residual:
        for m in self.modules():
          if isinstance(m, Bottleneck):
            nn.init.constant_(m.bn3.weight, 0)
          elif isinstance(m, BasicBlock):
            nn.init.constant_(m.bn2.weight, 0)

  # self.inplanes = 64          
  # self.layer1 = self._make_layer(block, 64, layers[0]) # 3
  def _make_layer(self, block, planes, blocks, stride = 1):
    downsample = None

    if stride != 1 or self.inplanes != planes * block.expansion:

      downsample = nn.Sequential(
          conv1x1(self.inplanes, planes * block.expansion, stride), # conv1x1(256, 512, 2)
          nn.BatchNorm2d(planes * block.expansion), # batchnorm2d(512)
      )

    layers = []
    layers.append(block(self.inplanes, planes, stride, downsample))
    # layers.append(BottleNeck(64, 64, 1, downsample))

    self.inplanes = planes * block.expansion # self.inplanes = 256

    for _ in range(1, blocks):
      layers.append(block(self.inplanes, planes)) # * 2

    return nn.Sequential(*layers)

  def forward(self, x):
    # x.shape = [batch, 3x32x32]
    x = self.conv1(x)
    # x.shape = [1, 16, 32, 32]
    x = self.bn1(x)
    x = self.relu(x)
    #x = self.maxpool(x)

    x = self.layer1(x)
    # x.shape = [1, 64, 32, 32]
    x = self.layer2(x)
    # x.shape = [1, 128, 32, 32]
    x = self.layer3(x)
    # x.shape = [1, 256, 16, 16]
    x = self.layer4(x)
    # x.shape = [1, 512, 8, 8]

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)

    return x

In [16]:
resnet50 = ResNet(Bottleneck, [3, 4, 6, 3], 10, True).to(device)

In [17]:
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [18]:
a = torch.Tensor(1, 3, 32, 32).to(device)
out = resnet50(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [19]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(resnet50.parameters(), lr = 0.1, momentum = 0.9, 
                            weight_decay = 5e-4)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)

In [20]:
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save:
        torch.save(net.state_dict(), "./model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

In [21]:
import os
os.mkdir('./model')

In [ ]:
print(len(trainloader))
epochs = 100

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    lr_sche.step()
    #Check Accuracy
    acc = acc_check(resnet50, testloader, epoch, save=1)
    
    

print('Finished Training')

196
[1,    30] loss: 2.091
[1,    60] loss: 1.937
[1,    90] loss: 1.796
[1,   120] loss: 1.730
[1,   150] loss: 1.647
[1,   180] loss: 1.590
Accuracy of the network on the 10000 test images: 43 %
[2,    30] loss: 1.507
[2,    60] loss: 1.443
[2,    90] loss: 1.373
[2,   120] loss: 1.359
[2,   150] loss: 1.299
[2,   180] loss: 1.254
Accuracy of the network on the 10000 test images: 56 %
[3,    30] loss: 1.160
[3,    60] loss: 1.153
[3,    90] loss: 1.133
[3,   120] loss: 1.066
[3,   150] loss: 1.048
[3,   180] loss: 1.040
Accuracy of the network on the 10000 test images: 65 %
[4,    30] loss: 0.979
[4,    60] loss: 0.961
[4,    90] loss: 0.951
[4,   120] loss: 0.968
[4,   150] loss: 0.901
[4,   180] loss: 0.902
Accuracy of the network on the 10000 test images: 68 %
[5,    30] loss: 0.867
[5,    60] loss: 0.850
[5,    90] loss: 0.859
[5,   120] loss: 0.856
[5,   150] loss: 0.835
[5,   180] loss: 0.796
Accuracy of the network on the 10000 test images: 72 %
[6,    30] loss: 0.777
[6,    6

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))